In [1]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

In [ ]:
# Tentukan path folder input dan output
input_folder = r'Data\wajah\crop\Fadh'  # Ganti dengan path folder gambar crop Anda
output_folder = r'Data\wajah\augment\Fadh'  # Ganti dengan path folder output

# Membuat folder output jika belum ada
os.makedirs(output_folder, exist_ok=True)

# Definisikan augmentasi menggunakan albumentations
augmenter = A.Compose([
    A.HorizontalFlip(p=0.5),  # Flip horizontal
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=25, val_shift_limit=15, p=0.5),  # Hue, Saturation, dan Brightness
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),  # Brightness dan Kontras
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),  # Exposure (RandomGamma)
    A.GaussianBlur(blur_limit=3, p=0.5),  # Blur
    A.ImageCompression(quality_lower=80, p=0.5)  # Noise (image compression)
])

def augment_and_save(image_path, relative_folder, filename, num_augmentations=10):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert ke RGB jika perlu

    # Membuat folder output untuk gambar ini
    target_folder = os.path.join(output_folder, relative_folder)
    os.makedirs(target_folder, exist_ok=True)

    # Menyimpan gambar asli
    original_image_path = os.path.join(target_folder, f"{filename}_original.jpg")
    cv2.imwrite(original_image_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))  # Convert kembali ke BGR untuk menyimpan

    for i in range(num_augmentations):
        augmented = augmenter(image=img)
        augmented_image = augmented['image']  # Hasil augmentasi tetap berupa numpy array

        # Menyimpan gambar yang telah di-augmentasi
        augmented_image_path = os.path.join(target_folder, f"{filename}_augmented_{i+1}.jpg")
        augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)  # Convert ke BGR untuk menyimpan
        cv2.imwrite(augmented_image_path, augmented_image_bgr)

# Loop melalui folder dan lakukan augmentasi dengan tqdm
for root, dirs, files in os.walk(input_folder):
    relative_folder = os.path.relpath(root, input_folder)  # Dapatkan folder relatif dari input_folder
    for file in tqdm(files, desc=f"Processing files in {relative_folder}"):
        if file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(root, file)
            filename, _ = os.path.splitext(file)
            augment_and_save(image_path, relative_folder, filename)

Processing files in .: 100%|██████████| 4/4 [00:02<00:00,  1.88it/s]


In [7]:
import os

# Tentukan folder input
input_folder = r'Data\wajah\augment'  # Ganti dengan path folder Anda

# Ekstensi file gambar yang didukung
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')

# Variabel untuk menghitung jumlah gambar
image_count = 0

# Loop melalui folder dan subfolder
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.lower().endswith(image_extensions):  # Periksa apakah file adalah gambar
            image_count += 1

print(f"Jumlah total gambar di folder '{input_folder}': {image_count}")


Jumlah total gambar di folder 'Data\wajah\augment': 1551
